In [1]:
import tensorflow as tf
from __future__ import print_function
from __future__ import division
import collections
import nltk
import numpy as np
from nltk.tokenize.casual import TweetTokenizer
from w266_common import utils, vocabulary
import time
import datetime
import os


/home/saulgrimaldo1/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import csv
tokenizer = TweetTokenizer()
x_data = []
x_contexts = []
labels = []
sentences  = []
contexts = []
with open('dta/merged_data_v2.csv', 'r') as csvfile:
    linereader = csv.reader(csvfile, delimiter = '|')
    for i, row in enumerate(linereader):
        if i == 0:
            continue
        sentence, context, sarcasm = row
        sentences.append(sentence)
        contexts.append(context)
        x_tokens = utils.canonicalize_words(tokenizer.tokenize(sentence))
        context_tokens = utils.canonicalize_words(tokenizer.tokenize(context))
        x_data.append(x_tokens)
        x_contexts.append(context_tokens)
        labels.append(int(sarcasm))


#rng = np.random.RandomState(5)
#rng.shuffle(x_data)  # in-place
#train_split_idx = int(0.7 * len(labels))
#test_split_idx  = int(0.9 * len(labels))

shuffle_indices = np.random.permutation(np.arange(len(labels)))
train_split_idx = int(0.7 * len(labels))
test_split_idx  = int(0.9 * len(labels))

train_indices = shuffle_indices[:train_split_idx]
validation_indices = shuffle_indices[train_split_idx:test_split_idx]
test_indices = shuffle_indices[test_split_idx:]


train_sentences = np.array(x_data)[train_indices]
train_contexts = np.array(x_contexts)[train_indices]
train_labels= np.array(labels)[train_indices] 
validation_sentences = np.array(x_data)[validation_indices]
validation_labels = np.array(labels)[validation_indices]
validation_contexts = train_contexts = np.array(x_contexts)[validation_indices]
test_sentences = np.array(x_data)[test_indices]  
test_contexts = train_contexts = np.array(x_contexts)[test_indices]
test_labels = np.array(labels)[test_indices]  

In [3]:
a = [2]*4
a[2] = 6
a

[2, 2, 6, 2]

In [4]:
def transform_labels(raw_label_set, size):
    label_set = []
    for label in raw_label_set:
        labels = [0] * size
        labels[label] = 1
        label_set.append(labels)
    return np.array(label_set)

expanded_train_labels = transform_labels(train_labels, 2)
expanded_validation_labels = transform_labels(validation_labels,2)
expanded_test_labels = transform_labels(test_labels,2)

In [5]:
a = [1,3,4,5,6,7,8,8,1,5,6,7]
a + ["<PADDING>"]*0

[1, 3, 4, 5, 6, 7, 8, 8, 1, 5, 6, 7]

In [6]:
class PaddingAndTruncating:
    def __init__(self, max_len):
        self.max_len = max_len
    
    def pad_or_truncate(self, sentence):
        sen_len = len(sentence)
        paddings = self.max_len - sen_len
        if paddings >=0:
            return sentence + ["<PADDING>"] * paddings
        return sentence[0:paddings]
        
PadAndTrunc = PaddingAndTruncating(10)
        
        
PadAndTrunc.pad_or_truncate(["the","angry","man","is","angry"])

['the',
 'angry',
 'man',
 'is',
 'angry',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>',
 '<PADDING>']

In [7]:
from tensorflow.contrib import learn

#max_len = max([len(sent) for sent  in train_sentences])
PadAndTrunc =PaddingAndTruncating(30)
train_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, train_sentences))
train_context_padded = list(map(PadAndTrunc.pad_or_truncate, train_contexts))
validation_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, validation_sentences))
validation_context_padded = list(map(PadAndTrunc.pad_or_truncate, validation_contexts))
test_sentences_padded = list(map(PadAndTrunc.pad_or_truncate, test_sentences))
test_context_padded = list(map(PadAndTrunc.pad_or_truncate, test_contexts))

vocab = vocabulary.Vocabulary(utils.flatten(list(train_sentences_padded) + list(train_context_padded)), 2500)
train_s = np.array(list(map(vocab.words_to_ids, train_sentences_padded)))
train_c = np.array(list(map(vocab.words_to_ids, train_context_padded)))
validation_s = np.array(list(map(vocab.words_to_ids, validation_sentences_padded)))
validation_c = np.array(list(map(vocab.words_to_ids, validation_context_padded)))
test_s = np.array(list(map(vocab.words_to_ids, test_sentences_padded)))
test_c = np.array(list(map(vocab.words_to_ids, test_context_padded)))
train_s

array([[  10,    4,    7, ...,    3,    3,    3],
       [  10,    4,    7, ...,   14,    3,    3],
       [2387,  246,   22, ..., 1691, 2391,    5],
       ..., 
       [  10,    4,    7, ...,    3,    3,    3],
       [   4,  130,    9, ...,  436,  849,  183],
       [   2,    2,   15, ...,    3,    3,    3]])

In [8]:
i = 3
("conv-maxpool-%s" % i)

'conv-maxpool-3'

In [9]:
class TextCNN(object):

    def __init__(
      self, sequence_length, num_classes, vocab_size, 
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x1 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x1")
        self.input_x2 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x2")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars1 = tf.nn.embedding_lookup(self.W, self.input_x1)
            self.embedded_chars_expanded1 = tf.expand_dims(self.embedded_chars1, -1)
            self.embedded_chars2 = tf.nn.embedding_lookup(self.W, self.input_x2)
            self.embedded_chars_expanded2 = tf.expand_dims(self.embedded_chars2, -1)

        # Create a convolution + avgpool layer for each filter size
        pooled_outputs = []

        for i, filter_size in enumerate(filter_sizes):
            with tf.variable_scope("conv-avgpool-%s" % i) as scope:
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]

                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
              
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded1,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h1 = tf.nn.tanh(tf.nn.bias_add(conv, b), name="tanh1")
               
                # Maxpooling over the outputs
                pooled = tf.nn.avg_pool(
                    h1,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                
                pooled_outputs.append(pooled)
                #pooled_outputs.append(pooled)
                scope.reuse_variables()
                 
                
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded2,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h2 = tf.nn.tanh(tf.nn.bias_add(conv, b), name="tanh2")
                # Maxpooling over the outputs
                pooled2 = tf.nn.avg_pool(
                    h2,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled2)

        # Combine all the pooled features

        num_filters_total = num_filters * len(filter_sizes) * 2

        self.h_pool = tf.concat(pooled_outputs, 1)

        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])


        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
        
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
           
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            self.labels = tf.argmax(self.input_y, 1)
            TP = tf.count_nonzero(self.predictions * self.labels)
            TN = tf.count_nonzero((self.predictions - 1) * (self.labels - 1))
            FP = tf.count_nonzero(self.predictions * (self.labels - 1))
            FN = tf.count_nonzero((self.predictions - 1) * self.labels)
            correct_predictions = tf.equal(self.predictions, self.labels)
            self.precision = TP / (TP + FP)
            self.recall = TP / (TP + FN)
            self.f1_score = 2 * self.precision * self.recall / (self.precision + self.recall)
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [10]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [11]:
learning_rate = 0.2

tf.flags.DEFINE_float("dev_sample_percentage", 0.5, "Percentage of the training data to use for validation")
tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 100, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "1", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 100, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.25, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0, "L2 regularization lambda (default: 0.0)")

# Training parameters
batch_size = 100
tf.flags.DEFINE_integer("batch_size", batch_size, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 500, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 20, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================


with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():

        cnn = TextCNN(
            sequence_length=train_s.shape[1],
            num_classes=2,
            vocab_size=2500,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)
        
         # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdagradOptimizer(learning_rate)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        
        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Write vocabulary
        #vocab_processor.save(os.path.join(out_dir, "vocab"))

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch1, x_batch2, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x1: x_batch1,
              cnn.input_x2: x_batch2,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch1, x_batch2, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x1: x_batch1,
              cnn.input_x2: x_batch2,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            #r 
            step, summaries, loss, accuracy, recall, precision, f1 = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy,cnn.recall, cnn.precision, cnn.f1_score],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}, rec {:g}, pre {:g}, f1 {:g}".format(time_str, step, loss, accuracy, recall, precision, f1))# recall, precision, f1))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(train_s, train_c,  expanded_train_labels)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch1, x_batch2, y_batch = zip(*batch)
            train_step(x_batch1, x_batch1, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(validation_s, validation_c, expanded_validation_labels, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))
print("\nTest Set:")
dev_step(test_s, test_c, expanded_test_labels, writer=dev_summary_writer)
print("")



Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=100
CHECKPOINT_EVERY=100
DEV_SAMPLE_PERCENTAGE=0.5
DROPOUT_KEEP_PROB=0.25
EMBEDDING_DIM=100
EVALUATE_EVERY=20
FILTER_SIZES=1
L2_REG_LAMBDA=0
LOG_DEVICE_PLACEMENT=False
NEGATIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.neg
NUM_CHECKPOINTS=5
NUM_EPOCHS=500
NUM_FILTERS=100
POSITIVE_DATA_FILE=./data/rt-polaritydata/rt-polarity.pos

INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-avgpool-0/W:0/grad/hist is illegal; using conv-avgpool-0/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-avgpool-0/W:0/grad/sparsity is illegal; using conv-avgpool-0/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-avgpool-0/b:0/grad/hist is illegal; using conv-avgpool-0/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-avgpool-0/b:0/gr

2018-04-16T00:29:28.081324: step 101, loss 0.395969, acc 0.79
2018-04-16T00:29:28.132633: step 102, loss 0.478528, acc 0.83
2018-04-16T00:29:28.184826: step 103, loss 0.682247, acc 0.68
2018-04-16T00:29:28.227021: step 104, loss 0.745632, acc 0.628205
2018-04-16T00:29:28.278786: step 105, loss 0.465558, acc 0.76
2018-04-16T00:29:28.334356: step 106, loss 0.471547, acc 0.79
2018-04-16T00:29:28.386162: step 107, loss 0.424685, acc 0.78
2018-04-16T00:29:28.428725: step 108, loss 0.409942, acc 0.820513
2018-04-16T00:29:28.479528: step 109, loss 0.366337, acc 0.85
2018-04-16T00:29:28.531568: step 110, loss 0.410731, acc 0.85
2018-04-16T00:29:28.588669: step 111, loss 0.448227, acc 0.83
2018-04-16T00:29:28.631631: step 112, loss 0.434256, acc 0.769231
2018-04-16T00:29:28.682648: step 113, loss 0.44486, acc 0.78
2018-04-16T00:29:28.734115: step 114, loss 0.553351, acc 0.73
2018-04-16T00:29:28.785181: step 115, loss 0.603414, acc 0.67
2018-04-16T00:29:28.830488: step 116, loss 0.634491, acc 0.

2018-04-16T00:29:34.926466: step 220, loss 0.52624, acc 0.766578, rec 0.678378, pre 0.814935, f1 0.740413

2018-04-16T00:29:35.005056: step 221, loss 0.289064, acc 0.87
2018-04-16T00:29:35.083312: step 222, loss 0.293436, acc 0.88
2018-04-16T00:29:35.163831: step 223, loss 0.333935, acc 0.9
2018-04-16T00:29:35.224012: step 224, loss 0.371988, acc 0.858974
2018-04-16T00:29:35.296728: step 225, loss 0.696238, acc 0.7
2018-04-16T00:29:35.364954: step 226, loss 0.659648, acc 0.76
2018-04-16T00:29:35.443581: step 227, loss 0.298231, acc 0.87
2018-04-16T00:29:35.506747: step 228, loss 0.318098, acc 0.884615
2018-04-16T00:29:35.568711: step 229, loss 0.326668, acc 0.87
2018-04-16T00:29:35.619943: step 230, loss 0.314196, acc 0.89
2018-04-16T00:29:35.675189: step 231, loss 0.336198, acc 0.86
2018-04-16T00:29:35.719272: step 232, loss 0.311228, acc 0.833333
2018-04-16T00:29:35.770284: step 233, loss 0.330532, acc 0.84
2018-04-16T00:29:35.820878: step 234, loss 0.36611, acc 0.84
2018-04-16T00:29

2018-04-16T00:29:41.681531: step 339, loss 0.181098, acc 0.93
2018-04-16T00:29:41.725815: step 340, loss 0.267112, acc 0.897436

Evaluation:
2018-04-16T00:29:41.826624: step 340, loss 0.847049, acc 0.582228, rec 0.932432, pre 0.543307, f1 0.686567

2018-04-16T00:29:41.877930: step 341, loss 0.210307, acc 0.92
2018-04-16T00:29:41.933063: step 342, loss 0.245581, acc 0.94
2018-04-16T00:29:41.984019: step 343, loss 0.162007, acc 0.94
2018-04-16T00:29:42.026934: step 344, loss 0.132163, acc 0.974359
2018-04-16T00:29:42.079612: step 345, loss 0.171047, acc 0.94
2018-04-16T00:29:42.131136: step 346, loss 0.200236, acc 0.94
2018-04-16T00:29:42.187900: step 347, loss 0.320558, acc 0.82
2018-04-16T00:29:42.230473: step 348, loss 0.492259, acc 0.807692
2018-04-16T00:29:42.283191: step 349, loss 0.309111, acc 0.84
2018-04-16T00:29:42.335131: step 350, loss 0.491845, acc 0.79
2018-04-16T00:29:42.387461: step 351, loss 0.271168, acc 0.87
2018-04-16T00:29:42.434433: step 352, loss 0.207394, acc 0.91

2018-04-16T00:29:48.401496: step 459, loss 0.213252, acc 0.92
2018-04-16T00:29:48.444032: step 460, loss 0.188731, acc 0.948718

Evaluation:
2018-04-16T00:29:48.545366: step 460, loss 0.550382, acc 0.751989, rec 0.854054, pre 0.703786, f1 0.771673

2018-04-16T00:29:48.600263: step 461, loss 0.133802, acc 0.97
2018-04-16T00:29:48.654434: step 462, loss 0.151451, acc 0.93
2018-04-16T00:29:48.706679: step 463, loss 0.151445, acc 0.95
2018-04-16T00:29:48.749342: step 464, loss 0.209987, acc 0.923077
2018-04-16T00:29:48.800755: step 465, loss 0.112172, acc 0.96
2018-04-16T00:29:48.852558: step 466, loss 0.174064, acc 0.95
2018-04-16T00:29:48.908064: step 467, loss 0.134817, acc 0.96
2018-04-16T00:29:48.950863: step 468, loss 0.158545, acc 0.935897
2018-04-16T00:29:49.002919: step 469, loss 0.121735, acc 0.97
2018-04-16T00:29:49.054274: step 470, loss 0.108759, acc 0.97
2018-04-16T00:29:49.105830: step 471, loss 0.0977494, acc 0.97
2018-04-16T00:29:49.151588: step 472, loss 0.137123, acc 0.9

...

2018-04-16T00:30:44.107792: step 1422, loss 0.00338191, acc 1
2018-04-16T00:30:44.161959: step 1423, loss 0.00393432, acc 1
2018-04-16T00:30:44.204091: step 1424, loss 0.0309795, acc 0.987179
2018-04-16T00:30:44.255836: step 1425, loss 0.034418, acc 0.99
2018-04-16T00:30:44.307122: step 1426, loss 0.00299861, acc 1
2018-04-16T00:30:44.362828: step 1427, loss 0.0056579, acc 1
2018-04-16T00:30:44.406520: step 1428, loss 0.0155989, acc 0.987179
2018-04-16T00:30:44.459054: step 1429, loss 0.00489312, acc 1
2018-04-16T00:30:44.511034: step 1430, loss 0.00550287, acc 1
2018-04-16T00:30:44.568521: step 1431, loss 0.00226709, acc 1
2018-04-16T00:30:44.611812: step 1432, loss 0.00237711, acc 1
2018-04-16T00:30:44.665745: step 1433, loss 0.00188631, acc 1
2018-04-16T00:30:44.718883: step 1434, loss 0.00267572, acc 1
2018-04-16T00:30:44.775273: step 1435, loss 0.0104046, acc 1
2018-04-16T00:30:44.818386: step 1436, loss 0.00476522, acc 1
2018-04-16T00:30:44.871246: step 1437, loss 0.00428417, acc

2018-04-16T00:30:50.909430: step 1542, loss 0.00345432, acc 1
2018-04-16T00:30:50.964849: step 1543, loss 0.00339374, acc 1
2018-04-16T00:30:51.007609: step 1544, loss 0.0162208, acc 0.987179
2018-04-16T00:30:51.059851: step 1545, loss 0.00886624, acc 1
2018-04-16T00:30:51.116474: step 1546, loss 0.0035078, acc 1
2018-04-16T00:30:51.168867: step 1547, loss 0.00292148, acc 1
2018-04-16T00:30:51.212354: step 1548, loss 0.00518706, acc 1
2018-04-16T00:30:51.352281: step 1549, loss 0.00105927, acc 1
2018-04-16T00:30:51.405031: step 1550, loss 0.00346291, acc 1
2018-04-16T00:30:51.457865: step 1551, loss 0.0103977, acc 0.99
2018-04-16T00:30:51.501076: step 1552, loss 0.0074642, acc 1
2018-04-16T00:30:51.557575: step 1553, loss 0.00320567, acc 1
2018-04-16T00:30:51.608764: step 1554, loss 0.00193645, acc 1
2018-04-16T00:30:51.660135: step 1555, loss 0.00225444, acc 1
2018-04-16T00:30:51.702765: step 1556, loss 0.00214543, acc 1
2018-04-16T00:30:51.756082: step 1557, loss 0.00589442, acc 1
20

2018-04-16T00:30:57.808434: step 1662, loss 0.00176495, acc 1
2018-04-16T00:30:57.859766: step 1663, loss 0.00316674, acc 1
2018-04-16T00:30:57.902402: step 1664, loss 0.00365181, acc 1
2018-04-16T00:30:57.954554: step 1665, loss 0.00184882, acc 1
2018-04-16T00:30:58.006848: step 1666, loss 0.00574128, acc 1
2018-04-16T00:30:58.062941: step 1667, loss 0.00485967, acc 1
2018-04-16T00:30:58.106197: step 1668, loss 0.00392346, acc 1
2018-04-16T00:30:58.159037: step 1669, loss 0.00366793, acc 1
2018-04-16T00:30:58.211756: step 1670, loss 0.00371983, acc 1
2018-04-16T00:30:58.263503: step 1671, loss 0.00394917, acc 1
2018-04-16T00:30:58.309460: step 1672, loss 0.00113689, acc 1
2018-04-16T00:30:58.361876: step 1673, loss 0.00170519, acc 1
2018-04-16T00:30:58.414106: step 1674, loss 0.00220214, acc 1
2018-04-16T00:30:58.465885: step 1675, loss 0.00199975, acc 1
2018-04-16T00:30:58.507962: step 1676, loss 0.0076415, acc 1
2018-04-16T00:30:58.563609: step 1677, loss 0.00251109, acc 1
2018-04-1

2018-04-16T00:31:04.701647: step 1782, loss 0.00271606, acc 1
2018-04-16T00:31:04.753795: step 1783, loss 0.00337688, acc 1
2018-04-16T00:31:04.796618: step 1784, loss 0.00214017, acc 1
2018-04-16T00:31:04.854240: step 1785, loss 0.00263886, acc 1
2018-04-16T00:31:04.909130: step 1786, loss 0.00145514, acc 1
2018-04-16T00:31:04.960840: step 1787, loss 0.00157672, acc 1
2018-04-16T00:31:05.002839: step 1788, loss 0.00178652, acc 1
2018-04-16T00:31:05.053952: step 1789, loss 0.00195738, acc 1
2018-04-16T00:31:05.105338: step 1790, loss 0.00114672, acc 1
2018-04-16T00:31:05.160428: step 1791, loss 0.00629129, acc 1
2018-04-16T00:31:05.203391: step 1792, loss 0.00167803, acc 1
2018-04-16T00:31:05.255179: step 1793, loss 0.00379324, acc 1
2018-04-16T00:31:05.307245: step 1794, loss 0.00237796, acc 1
2018-04-16T00:31:05.359324: step 1795, loss 0.00138722, acc 1
2018-04-16T00:31:05.406491: step 1796, loss 0.00221096, acc 1
2018-04-16T00:31:05.458106: step 1797, loss 0.0024289, acc 1
2018-04-1

Saved model checkpoint to /home/saulgrimaldo1/sarcasm_detection_w266/runs/1523838561/checkpoints/model-1900

2018-04-16T00:31:11.542394: step 1901, loss 0.00147693, acc 1
2018-04-16T00:31:11.594229: step 1902, loss 0.00256313, acc 1
2018-04-16T00:31:11.650573: step 1903, loss 0.00290862, acc 1
2018-04-16T00:31:11.696927: step 1904, loss 0.00155127, acc 1
2018-04-16T00:31:11.749674: step 1905, loss 0.00152615, acc 1
2018-04-16T00:31:11.802906: step 1906, loss 0.00204615, acc 1
2018-04-16T00:31:11.855989: step 1907, loss 0.00231597, acc 1
2018-04-16T00:31:11.905412: step 1908, loss 0.00130831, acc 1
2018-04-16T00:31:11.957751: step 1909, loss 0.00192467, acc 1
2018-04-16T00:31:12.009072: step 1910, loss 0.00129459, acc 1
2018-04-16T00:31:12.060943: step 1911, loss 0.00110215, acc 1
2018-04-16T00:31:12.102880: step 1912, loss 0.00273609, acc 1
2018-04-16T00:31:12.158682: step 1913, loss 0.000872197, acc 1
2018-04-16T00:31:12.210786: step 1914, loss 0.00158883, acc 1
2018-04-16T00:31:12.26